<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/04_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語ベクトル
* いわゆるword2vec。
 * https://arxiv.org/abs/1301.3781
 * https://en.wikipedia.org/wiki/Word2vec
* 単語をベクトルとして表現したもの。
 * 単語埋め込み、単語分散表現、などとも言われる。
* 意味が近い単語はベクトルとしても近くなるように、作成されている。


* 今回はランタイムのタイプでGPUを選んでおいてください。

## 使いみち
* 単語どうしの類似度評価に使う。
* 文書どうしの類似度評価にも使う。
 * 文書をbag-of-wordsモデルによってベクトル表現することは、最近はあまりない。
 * 単語を使って文書のベクトル表現を作る。
 * とはいえ、word2vecも最近はあまり使われない。
 * 今は言語モデルを使う。何回か後の授業で説明する。

## 単語ベクトルを作るアルゴリズム
* 説明は割愛する。

In [ ]:
!pip install datasets

## WRIME: 主観と客観の感情分析データセット
* https://github.com/ids-cv/wrime

In [ ]:
from datasets import load_dataset

dataset = load_dataset("shunk031/wrime", "ver2")

In [ ]:
import numpy as np

tags = ["train", "validation", "test"]

texts = {}
labels = {}
for tag in tags:
  texts[tag] = dataset[tag]["sentence"]
  labels[tag] = [item["sentiment"] for item in dataset[tag]["avg_readers"]]
  labels[tag] = np.array(labels[tag])

## spaCyの単語ベクトル
* 今回は英語テキストのみ。
* 小規模のモデル（名前が__`_sm`__で終わるモデル）は単語ベクトルを含まない。
* 大規模モデルはダウンロードに時間がかかる。
* そのため、中規模モデルをインストールする。

### spaCyの中規模モデルをダウンロード
* https://spacy.io/models/ja#ja_core_news_md

In [ ]:
!python -m spacy download ja_core_news_md

* 全テキストのベクトル化
 * 数分待つ。

In [ ]:
from tqdm import tqdm
import numpy as np
import spacy

nlp = spacy.load('ja_core_news_md')

X = {}
for tag in tags:
  X[tag] = []
  for text in tqdm(texts[tag]):
    tokens = nlp(text)
    X[tag].append(tokens.vector)
  X[tag] = np.array(X[tag])

In [ ]:
X["validation"].shape

In [ ]:
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'wb') as f:
    np.save(f, X[tag])
  with open(f'wrime_{tag}_label.npy', 'wb') as f:
    np.save(f, labels[tag])

## 単語ベクトルを使った文書分類

In [ ]:
X = {}
labels = {}
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'rb') as f:
    X[tag] = np.load(f)
  with open(f'wrime_{tag}_label.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
X["train"].shape

In [ ]:
labels["train"].shape

In [ ]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

In [ ]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

## BERTによる文書のベクトル化
* BERTの説明はしない。とりあえず使う。
* BERTを単なるエンコーダとして使う。
 * fine-tuningはしない。

In [ ]:
!pip install -q transformers fugashi[unidic-lite]

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("cl-tohoku/bert-base-japanese-v3")

In [ ]:
X = {}
for tag in tags:
  X[tag] = embedder.encode(texts[tag])

In [ ]:
import numpy as np

for tag in tags:
  with open(f'wrime_{tag}_bert_vec.npy', 'wb') as f:
    np.save(f, X[tag])

In [ ]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

In [ ]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

# 課題
* 上で実行した感情分析の性能を上げてください。
* チューニングが済んだら、テストセットでscoreを計算してください。